In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import time
import openpyxl
import re

In [2]:
# Initialize an Excel workbook and worksheet
wb = openpyxl.Workbook()
ws = wb.active

# Setup column headers in the Excel sheet using a list
headers = ["Job Title", "Job Link", "Company Name", "Industry", "Company Size", "Work Area", 
           "Required Work Experience", "Educational Requirements", "Number of Applicants", 
           "Salary", "Payment Method", "Salary Lower Bound", "Salary Upper Bound"]

ws.append(headers)

In [3]:
# Define your keyword as a variable
keyword = "data analyst 數據分析"

# Convert spaces to URL encoding for the keyword
formatted_keyword = keyword.replace(" ", "%20")

# Define the base URL with a placeholder for the page number
base_url = f"https://www.104.com.tw/jobs/search/?ro=0&kwop=1&keyword={formatted_keyword}&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=12&asc=0&page={{page}}&mode=s&jobsource=index_s&langFlag=0&langStatus=0&recommendJob=1&hotJob=1"

In [4]:
# Start timing the script
start_time = time.time()

page = 1

while True:
    url = base_url.format(page=page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    print('---------------------------------------------------')
    print(f"Currently reading page {page}")
    print('---------------------------------------------------')
    
    # Extract all job listings on the current page
    jobs = soup.find_all("article", class_="b-block--top-bord job-list-item b-clearfix js-job-item")
    
    # If no jobs are found, break the loop
    if not jobs:
        print("No more job listings found. Exiting the loop.")
        break

    for job in jobs:
        job_title = job.a.text.strip()
        print(f"Job Title: {job_title}")
        
        job_link = "https:" + job.a["href"].strip()
        print(f"Job Link: {job_link}")
        
        company_name = job.select("li")[1].text.strip()
        print(f"Company Name: {company_name}")

        # Find all <a> tags with class 'b-tag--default' within the 'job-list-tag b-content' div
        a_tags = job.find_all("a", class_="b-tag--default")
        company_size = "Not specified"
        for tag in a_tags:
            if "員工" in tag.text:
                company_size = tag.text.strip()
                break
        print(f"Company Size: {company_size}")
        
        industry_tag = job.find("div", class_="b-divide").find_next_sibling("li")
        industry = industry_tag.text.strip() if industry_tag else "Not specified"
        print(f"Industry: {industry}")

        # Locate the salary information container
        salary_div = job.find('div', class_="job-list-tag b-content")

        # Initialize salary-related variables
        salary = ""
        payment_method = ""
        salary_lower_bound = ""
        salary_upper_bound = ""

        if salary_div:
            # Check for salary-related keywords in <a> tags (e.g., "時薪", "月薪", "年薪")
            salary_details = salary_div.a
            if salary_details and any(keyword in salary_details.text for keyword in ["時薪", "月薪", "年薪"]):
                salary = salary_details.text.strip()
                payment_method = salary[:2]
                salary_numbers = ''.join([char for char in salary if char.isdigit() or char == "~"])
                salary_lower_bound, salary_upper_bound = (salary_numbers.split('~') + [None, None])[:2]

            # Check if "面議" (negotiable) is present in <span> tags
            elif salary_div.span and "面議" in salary_div.span.text:
                salary = "待遇面議"
                payment_method = "待遇面議"
                salary_lower_bound = "N/A"
                salary_upper_bound = "N/A"

        # Print the extracted information
        print(f"Salary: {salary}")
        print(f"Payment Method: {payment_method}")
        print(f"Salary Lower Bound: {salary_lower_bound}")
        print(f"Salary Upper Bound: {salary_upper_bound}")

        ul_tag = job.find("ul", class_="b-list-inline b-clearfix job-list-intro b-content")
        if ul_tag:
            li_tags = ul_tag.find_all("li")
            work_area = li_tags[0].text.strip() if len(li_tags) > 0 and li_tags[0] is not None else "Not specified"
            work_exp = li_tags[1].text.strip() if len(li_tags) > 1 and li_tags[1] is not None else "Not specified"
            education = li_tags[2].text.strip() if len(li_tags) > 2 and li_tags[2] is not None else "Not specified"
        else:
            work_area = "Not specified"
            work_exp = "Not specified"
            education = "Not specified"
        
        print(f"Work Area: {work_area}")
        print(f"Work Experience: {work_exp}")
        print(f"Education: {education}")

        num_apply_tag = job.find('a', class_='b-link--gray gtm-list-apply')
        num_apply = "Not specified"
        for tag in num_apply_tag:
            if "人" in tag.text:
                num_apply = tag.text.strip()
                break
        print(f"Applicants Number: {num_apply}")
        print("---------------------This is a separator---------------------")

        ws.append([job_title, job_link, company_name, industry, company_size, work_area, work_exp, education, num_apply, salary, payment_method, salary_lower_bound, salary_upper_bound])
        
    
    page += 1
    sleep(1)  # Pause between requests to be polite to the server

print("Scraping complete.")

wb.save("data_analyst_job_vacancies.xlsx")

# Calculate and print the total execution time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total execution time: {elapsed_time:.2f} seconds")

---------------------------------------------------
Currently reading page 1
---------------------------------------------------
Job Title: Clinical Trial Data Analyst 臨床試驗數據分析師
Job Link: https://www.104.com.tw/job/8gukx?jobsource=index_s
Company Name: 醫乘智慧有限公司
Company Size: Not specified
Industry: 其它軟體及網路相關業
Salary: 待遇面議
Payment Method: 待遇面議
Salary Lower Bound: N/A
Salary Upper Bound: N/A
Work Area: 台北市信義區
Work Experience: 經歷不拘
Education: 大學
Applicants Number: 0~5人應徵
---------------------This is a separator---------------------
Job Title: Data Analyst 數據分析師(資訊處電腦組)
Job Link: https://www.104.com.tw/job/6vdp2?jobsource=index_s
Company Name: 臺灣電視事業股份有限公司
Company Size: 員工525人
Industry: 電視業
Salary: 月薪35,000元以上
Payment Method: 月薪
Salary Lower Bound: 35000
Salary Upper Bound: None
Work Area: 台北市松山區
Work Experience: 經歷不拘
Education: 大學
Applicants Number: 6~10人應徵
---------------------This is a separator---------------------
Job Title: 【GenApe生成猿】產品企劃師、數據分析師（Product Plan & Data Analyst）
Job Link

<br>

### Data Cleaning


In [5]:
import pandas as pd

In [6]:
df_job = pd.read_excel('data_analyst_job_vacancies.xlsx')
df_job.head(3)

,Job Title,Job Link,Company Name,Industry,Company Size,Work Area,Required Work Experience,Educational Requirements,Number of Applicants,Salary,Payment Method,Salary Lower Bound,Salary Upper Bound
0,Clinical Trial Data Analyst 臨床試驗數據分析師,https://www.104.com.tw/job/8gukx?jobsource=ind...,醫乘智慧有限公司,其它軟體及網路相關業,Not specified,台北市信義區,經歷不拘,大學,0~5人應徵,待遇面議,待遇面議,NaN,NaN
1,Data Analyst 數據分析師(資訊處電腦組),https://www.104.com.tw/job/6vdp2?jobsource=ind...,臺灣電視事業股份有限公司,電視業,員工525人,台北市松山區,經歷不拘,大學,6~10人應徵,"月薪35,000元以上",月薪,35000.0,NaN
2,【GenApe生成猿】產品企劃師、數據分析師（Product Plan & Data Ana...,https://www.104.com.tw/job/7ga0s?jobsource=ind...,玩構網路科技有限公司,其它軟體及網路相關業,員工15人,高雄市楠梓區,經歷不拘,大學,0~5人應徵,"月薪38,000~60,000元",月薪,38000.0,60000.0


In [ ]:
# # Convert the DataFrame to a CSV file
# df_job.to_csv('data_analyst_job_vacancies.csv', index=False)

In [ ]:
# df_job.head(3)

In [ ]:
df_job.info()

In [7]:
# Convert relevant columns to string
df_job['Job Title'] = df_job['Job Title'].astype('string')
df_job['Job Link'] = df_job['Job Link'].astype('string')
df_job['Company Name'] = df_job['Company Name'].astype('string')
df_job['Industry'] = df_job['Industry'].astype('string')
df_job['Work Area'] = df_job['Work Area'].astype('string')
df_job['Required Work Experience'] = df_job['Required Work Experience'].astype('string')
df_job['Educational Requirements'] = df_job['Educational Requirements'].astype('string')
df_job['Payment Method'] = df_job['Payment Method'].astype('string')

# Convert 'Company Size' and 'Number of Applicants' to int after cleaning (removing non-numeric symbols)
df_job['Company Size'] = df_job['Company Size'].str.extract('(\d+)').astype(float).astype('Int64')
df_job['Number of Applicants'] = df_job['Number of Applicants'].str.extract('(\d+)').astype(float).astype('Int64')

# Convert 'Salary Lower Bound' and 'Salary Upper Bound' to integer
df_job['Salary Lower Bound'] = df_job['Salary Lower Bound'].fillna(0).astype(int)
df_job['Salary Upper Bound'] = df_job['Salary Upper Bound'].fillna(0).astype(int)

In [8]:
df_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Job Title                 1111 non-null   string
 1   Job Link                  1111 non-null   string
 2   Company Name              1111 non-null   string
 3   Industry                  1111 non-null   string
 4   Company Size              860 non-null    Int64 
 5   Work Area                 1111 non-null   string
 6   Required Work Experience  1111 non-null   string
 7   Educational Requirements  1111 non-null   string
 8   Number of Applicants      1111 non-null   Int64 
 9   Salary                    1111 non-null   object
 10  Payment Method            1111 non-null   string
 11  Salary Lower Bound        1111 non-null   int32 
 12  Salary Upper Bound        1111 non-null   int32 
dtypes: Int64(2), int32(2), object(1), string(8)
memory usage: 106.5+ KB


In [9]:
df_job.head(10)

,Job Title,Job Link,Company Name,Industry,Company Size,Work Area,Required Work Experience,Educational Requirements,Number of Applicants,Salary,Payment Method,Salary Lower Bound,Salary Upper Bound
0,Clinical Trial Data Analyst 臨床試驗數據分析師,https://www.104.com.tw/job/8gukx?jobsource=ind...,醫乘智慧有限公司,其它軟體及網路相關業,<NA>,台北市信義區,經歷不拘,大學,0,待遇面議,待遇面議,0,0
1,Data Analyst 數據分析師(資訊處電腦組),https://www.104.com.tw/job/6vdp2?jobsource=ind...,臺灣電視事業股份有限公司,電視業,525,台北市松山區,經歷不拘,大學,6,"月薪35,000元以上",月薪,35000,0
2,【GenApe生成猿】產品企劃師、數據分析師（Product Plan & Data Ana...,https://www.104.com.tw/job/7ga0s?jobsource=ind...,玩構網路科技有限公司,其它軟體及網路相關業,15,高雄市楠梓區,經歷不拘,大學,0,"月薪38,000~60,000元",月薪,38000,60000
3,採購數據分析師 Procurement Data Analyst,https://www.104.com.tw/job/7rppv?jobsource=ind...,美時化學製藥股份有限公司,藥品製造業,750,台中市西屯區,1年以上,專科,6,待遇面議,待遇面議,0,0
4,金融數據分析師 Data Analyst,https://www.104.com.tw/job/7pxqm?jobsource=ind...,烽泰科技有限公司,其他金融及輔助業,140,台北市大同區,1年以上,大學,11,待遇面議,待遇面議,0,0
5,數據分析工程師 Data Analysis Engineer,https://www.104.com.tw/job/8bdtc?jobsource=ind...,浩騰媒體股份有限公司,廣告行銷公關業,200,台北市中山區,3年以上,大學,0,待遇面議,待遇面議,0,0
6,Data Analyst 數據分析師,https://www.104.com.tw/job/7t7ba?jobsource=ind...,欒軒管理顧問有限公司,工商顧問服務業,30,台北市松山區,3年以上,大學,11,"月薪50,000元以上",月薪,50000,0
7,Investment Data Assistant 投資數據分析助理,https://www.104.com.tw/job/8cup5?jobsource=ind...,大拇哥證券投資顧問股份有限公司,證券及期貨業,50,台北市大安區,1年以上,大學,6,"月薪40,000~55,000元",月薪,40000,55000
8,商業數據分析師(BUSINESS ANALYST),https://www.104.com.tw/job/6i8ue?jobsource=ind...,艾思網絡股份有限公司,網際網路相關業,30,台北市中正區,2年以上,碩士,6,待遇面議,待遇面議,0,0
9,【海外通路】(外派)數據分析師 Data Analyst_I00015658,https://www.104.com.tw/job/8a63d?jobsource=ind...,國泰世華商業銀行股份有限公司_人力資源部,銀行業,10000,越南,2年以上,學歷不拘,6,待遇面議,待遇面議,0,0


In [10]:
# Fill 'Salary Lower Bound' with 40000 where 'Salary' is '待遇面議'
df_job.loc[df_job['Salary'] == '待遇面議', 'Salary Lower Bound'] = 40000

In [11]:
df_job.head(3)

,Job Title,Job Link,Company Name,Industry,Company Size,Work Area,Required Work Experience,Educational Requirements,Number of Applicants,Salary,Payment Method,Salary Lower Bound,Salary Upper Bound
0,Clinical Trial Data Analyst 臨床試驗數據分析師,https://www.104.com.tw/job/8gukx?jobsource=ind...,醫乘智慧有限公司,其它軟體及網路相關業,<NA>,台北市信義區,經歷不拘,大學,0,待遇面議,待遇面議,40000,0
1,Data Analyst 數據分析師(資訊處電腦組),https://www.104.com.tw/job/6vdp2?jobsource=ind...,臺灣電視事業股份有限公司,電視業,525,台北市松山區,經歷不拘,大學,6,"月薪35,000元以上",月薪,35000,0
2,【GenApe生成猿】產品企劃師、數據分析師（Product Plan & Data Ana...,https://www.104.com.tw/job/7ga0s?jobsource=ind...,玩構網路科技有限公司,其它軟體及網路相關業,15,高雄市楠梓區,經歷不拘,大學,0,"月薪38,000~60,000元",月薪,38000,60000


In [14]:
def convert_salary_to_monthly(row):
    if row['Payment Method'] == '月薪':
        return row['Salary Lower Bound']
    elif row['Payment Method'] == '年薪':
        return row['Salary Lower Bound'] / 12
    else:
        return None

df_job['Monthly Salary'] = df_job.apply(convert_salary_to_monthly, axis=1)

In [15]:
df_job.head(3)

,Job Title,Job Link,Company Name,Industry,Company Size,Work Area,Required Work Experience,Educational Requirements,Number of Applicants,Salary,Payment Method,Salary Lower Bound,Salary Upper Bound,Monthly Salary
0,Clinical Trial Data Analyst 臨床試驗數據分析師,https://www.104.com.tw/job/8gukx?jobsource=ind...,醫乘智慧有限公司,其它軟體及網路相關業,<NA>,台北市信義區,經歷不拘,大學,0,待遇面議,待遇面議,40000,0,NaN
1,Data Analyst 數據分析師(資訊處電腦組),https://www.104.com.tw/job/6vdp2?jobsource=ind...,臺灣電視事業股份有限公司,電視業,525,台北市松山區,經歷不拘,大學,6,"月薪35,000元以上",月薪,35000,0,35000.0
2,【GenApe生成猿】產品企劃師、數據分析師（Product Plan & Data Ana...,https://www.104.com.tw/job/7ga0s?jobsource=ind...,玩構網路科技有限公司,其它軟體及網路相關業,15,高雄市楠梓區,經歷不拘,大學,0,"月薪38,000~60,000元",月薪,38000,60000,38000.0


In [16]:
df_job.to_excel('data_analyst_job_vacancies_cleaned.xlsx', index=False)

<br>
<br>

### Postal Coordinate

In [17]:
df_postal_codes = pd.read_excel('3碼郵遞區號與行政區中心點經緯度對照表.xlsx')
df_postal_codes.head(3)

,行政區名,_x0033_碼郵遞區號,中心點經度,中心點緯度,TGOS_URL
0,臺北市中正區,100,121.519884,25.032405,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
1,臺北市大同區,103,121.513042,25.063424,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
2,臺北市中山區,104,121.538160,25.069699,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...


In [18]:
df_postal_codes['行政區名'] = df_postal_codes['行政區名'].str.replace('臺', '台')
df_postal_codes.head(3)

,行政區名,_x0033_碼郵遞區號,中心點經度,中心點緯度,TGOS_URL
0,台北市中正區,100,121.519884,25.032405,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
1,台北市大同區,103,121.513042,25.063424,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
2,台北市中山區,104,121.538160,25.069699,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...


In [30]:
df_job_latlon = pd.merge(df_job, df_postal_codes, left_on='Work Area', right_on='行政區名', how='left')

df_job_latlon.head()

,Job Title,Job Link,Company Name,Industry,Company Size,Work Area,Required Work Experience,Educational Requirements,Number of Applicants,Salary,Payment Method,Salary Lower Bound,Salary Upper Bound,Monthly Salary,行政區名,_x0033_碼郵遞區號,中心點經度,中心點緯度,TGOS_URL
0,Clinical Trial Data Analyst 臨床試驗數據分析師,https://www.104.com.tw/job/8gukx?jobsource=ind...,醫乘智慧有限公司,其它軟體及網路相關業,<NA>,台北市信義區,經歷不拘,大學,0,待遇面議,待遇面議,40000,0,NaN,台北市信義區,110.0,121.571670,25.030621,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
1,Data Analyst 數據分析師(資訊處電腦組),https://www.104.com.tw/job/6vdp2?jobsource=ind...,臺灣電視事業股份有限公司,電視業,525,台北市松山區,經歷不拘,大學,6,"月薪35,000元以上",月薪,35000,0,35000.0,台北市松山區,105.0,121.557588,25.059991,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
2,【GenApe生成猿】產品企劃師、數據分析師（Product Plan & Data Ana...,https://www.104.com.tw/job/7ga0s?jobsource=ind...,玩構網路科技有限公司,其它軟體及網路相關業,15,高雄市楠梓區,經歷不拘,大學,0,"月薪38,000~60,000元",月薪,38000,60000,38000.0,高雄市楠梓區,811.0,120.300758,22.721100,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
3,採購數據分析師 Procurement Data Analyst,https://www.104.com.tw/job/7rppv?jobsource=ind...,美時化學製藥股份有限公司,藥品製造業,750,台中市西屯區,1年以上,專科,6,待遇面議,待遇面議,40000,0,NaN,台中市西屯區,407.0,120.627013,24.183089,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...
4,金融數據分析師 Data Analyst,https://www.104.com.tw/job/7pxqm?jobsource=ind...,烽泰科技有限公司,其他金融及輔助業,140,台北市大同區,1年以上,大學,11,待遇面議,待遇面議,40000,0,NaN,台北市大同區,103.0,121.513042,25.063424,http://tgos.nat.gov.tw/tgos/Web/MetaData/TGOS_...


In [31]:
df_job_latlon.columns

Index(['Job Title', 'Job Link', 'Company Name', 'Industry', 'Company Size',
       'Work Area', 'Required Work Experience', 'Educational Requirements',
       'Number of Applicants', 'Salary', 'Payment Method',
       'Salary Lower Bound', 'Salary Upper Bound', 'Monthly Salary', '行政區名',
       '_x0033_碼郵遞區號', '中心點經度', '中心點緯度', 'TGOS_URL'],
      dtype='object')

In [33]:
df_job_latlon = df_job_latlon[['Job Title', 'Job Link', 'Company Name', 'Industry', 'Company Size',
                               'Work Area', 'Required Work Experience', 'Educational Requirements', 'Number of Applicants',
                               'Salary', 'Payment Method', 'Salary Lower Bound', 'Salary Upper Bound', 'Monthly Salary',
                               '中心點經度', '中心點緯度']]

In [34]:
df_job_latlon.head()

,Job Title,Job Link,Company Name,Industry,Company Size,Work Area,Required Work Experience,Educational Requirements,Number of Applicants,Salary,Payment Method,Salary Lower Bound,Salary Upper Bound,Monthly Salary,中心點經度,中心點緯度
0,Clinical Trial Data Analyst 臨床試驗數據分析師,https://www.104.com.tw/job/8gukx?jobsource=ind...,醫乘智慧有限公司,其它軟體及網路相關業,<NA>,台北市信義區,經歷不拘,大學,0,待遇面議,待遇面議,40000,0,NaN,121.571670,25.030621
1,Data Analyst 數據分析師(資訊處電腦組),https://www.104.com.tw/job/6vdp2?jobsource=ind...,臺灣電視事業股份有限公司,電視業,525,台北市松山區,經歷不拘,大學,6,"月薪35,000元以上",月薪,35000,0,35000.0,121.557588,25.059991
2,【GenApe生成猿】產品企劃師、數據分析師（Product Plan & Data Ana...,https://www.104.com.tw/job/7ga0s?jobsource=ind...,玩構網路科技有限公司,其它軟體及網路相關業,15,高雄市楠梓區,經歷不拘,大學,0,"月薪38,000~60,000元",月薪,38000,60000,38000.0,120.300758,22.721100
3,採購數據分析師 Procurement Data Analyst,https://www.104.com.tw/job/7rppv?jobsource=ind...,美時化學製藥股份有限公司,藥品製造業,750,台中市西屯區,1年以上,專科,6,待遇面議,待遇面議,40000,0,NaN,120.627013,24.183089
4,金融數據分析師 Data Analyst,https://www.104.com.tw/job/7pxqm?jobsource=ind...,烽泰科技有限公司,其他金融及輔助業,140,台北市大同區,1年以上,大學,11,待遇面議,待遇面議,40000,0,NaN,121.513042,25.063424


In [35]:
df_job_latlon['Work Area'].value_counts()

Work Area
台北市信義區    195
台北市內湖區    130
台北市松山區    110
台北市中山區     93
台北市大安區     84
         ... 
台中市中區       1
高雄市岡山區      1
台中市北屯區      1
高雄市大寮區      1
屏東縣枋寮鄉      1
Name: count, Length: 85, dtype: int64

In [36]:
# Function to split Work Area based on different patterns
def split_work_area(work_area):
    # Special case for "新竹市" being incorrectly labeled when it should be "新竹縣"
    if work_area == "新竹市":
        city = "新竹縣"
        district = "新竹市"
    elif '市' in work_area and '區' in work_area:
        # Case: City and District
        city_district = work_area.split('市', 1)
        city = city_district[0] + '市'
        district = city_district[1]  # District already contains "區"
    elif '縣' in work_area and ('鎮' in work_area or '鄉' in work_area):
        # Case: County and Town/Township
        city_district = work_area.split('縣', 1)
        city = city_district[0] + '縣'
        district = city_district[1]  # District is either "鎮" or "鄉"
    elif '縣' in work_area and '市' in work_area:
        # Case: County and City (e.g., 宜蘭縣宜蘭市)
        city_district = work_area.split('縣', 1)
        city = city_district[0] + '縣'
        district = city_district[1]  # District is "市"
    elif '市' in work_area:
        # Case: Only City (without District)
        city = work_area  # The entire work_area is the city
        district = None
    else:
        # Default case: No recognizable pattern
        city = work_area
        district = None
    return pd.Series([city, district])

In [38]:
df_job_latlon[['City', 'District']] = df_job_latlon['Work Area'].apply(split_work_area)

C:\Users\USER\AppData\Local\Temp\ipykernel_23648\720390603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_latlon[['City', 'District']] = df_job_latlon['Work Area'].apply(split_work_area)


In [39]:
# Display the updated DataFrame to verify the changes
print(df_job_latlon[['Work Area', 'City', 'District']].head(20))

   Work Area City District
0     台北市信義區  台北市      信義區
1     台北市松山區  台北市      松山區
2     高雄市楠梓區  高雄市      楠梓區
3     台中市西屯區  台中市      西屯區
4     台北市大同區  台北市      大同區
5     台北市中山區  台北市      中山區
6     台北市松山區  台北市      松山區
7     台北市大安區  台北市      大安區
8     台北市中正區  台北市      中正區
9         越南   越南     None
10    台北市中正區  台北市      中正區
11    台北市松山區  台北市      松山區
12    台北市信義區  台北市      信義區
13    台北市中山區  台北市      中山區
14    台北市中山區  台北市      中山區
15    台北市信義區  台北市      信義區
16    台北市大安區  台北市      大安區
17    台北市南港區  台北市      南港區
18    台北市中山區  台北市      中山區
19    台北市信義區  台北市      信義區


In [40]:
taiwan_cities_counties = [
    "台北市", "新北市", "桃園市", "台中市", "台南市", "高雄市", 
    "基隆市", "新竹市", "嘉義市", "新竹縣", "苗栗縣", "彰化縣", 
    "南投縣", "雲林縣", "嘉義縣", "屏東縣", "宜蘭縣", "花蓮縣", 
    "台東縣", "澎湖縣", "金門縣", "連江縣"]

In [41]:
print(len(df_job_latlon))
df_job_latlon = df_job_latlon[df_job_latlon['City'].isin(taiwan_cities_counties)]
print(len(df_job_latlon))

1111
1097


In [43]:
df_job_latlon.to_excel('data_analyst_job_vacancies_cleaned_with_latlon.xlsx', index=False)